El análisis discriminante lineal (LDA) es una técnica de reducción de dimensionalidad. Como su nombre lo indica, las técnicas de reducción de la dimensionalidad reducen el número de dimensiones (es decir, variables) en un conjunto de datos mientras retienen la mayor cantidad de información posible.

Por ejemplo, suponga que graficamos la relación entre dos variables donde cada color representa una clase diferente.

![](https://miro.medium.com/max/700/1*o2TKovc_lkJ9_ISxZxrbog.png)

Si quisiéramos reducir el número de dimensiones a 1, un enfoque sería proyectar todo en el eje x.

![](https://miro.medium.com/max/700/1*5lugB_AavKEr3ghDGC6hOA.png)

![](https://miro.medium.com/max/635/1*Z202fIHoHkW5KhxxXcQ8jA.png)

Esto es malo porque ignora cualquier información útil proporcionada por la segunda función. Por otro lado, el Análisis Discriminante Lineal, o LDA, usa la información de ambas características para crear un nuevo eje y proyecta los datos en el nuevo eje de tal manera que minimiza la varianza y maximiza la distancia entre las medias del dos clases.

![](https://miro.medium.com/max/700/1*Fz3JQ80No5Nnbap28EGRTg.png)

![](https://miro.medium.com/max/700/1*5lhckC2RQzq28zNL7WtU5A.png)

![](https://miro.medium.com/max/700/1*W48aQ0LkZ5dm1_uow6FD2w.png)

Veamos cómo podemos implementar el análisis discriminante lineal desde cero con Python. Para comenzar, importe las siguientes bibliotecas.












In [ ]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

En este notebook , trabajaremos con el conjunto de datos de wine que se puede obtener del repositorio de aprendizaje automático de UCI.

In [ ]:
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Categorical.from_codes(wine.target, wine.target_names)
X.shape

Las variables se componen de varias características como el magnesio y el contenido de alcohol del vino.

In [ ]:
X.head()

Hay 3 tipos diferentes de vino.

In [ ]:
wine.target_names

Creamos un DataFrame que contiene tanto las características como las clases.

In [ ]:
df = X.join(pd.Series(y, name='class'))

El análisis discriminante lineal se puede dividir en los siguientes pasos:

> Calcule las matrices de dispersión intraclase y entre clases

> Calcule los autovectores y los autovalores correspondientes para las matrices de dispersión.

> Ordene los valores propios y seleccione el k superior

> Cree una nueva matriz que contenga vectores propios que se asignen a los k valores propios

> Obtenga las nuevas características (es decir, componentes LDA) tomando el producto escalar de los datos y la matriz del paso anterior.

Para cada clase, creamos un vector con las medias de cada característica.


In [ ]:
class_feature_means = pd.DataFrame(columns=wine.target_names)
for c, rows in df.groupby('class'):
    class_feature_means[c] = rows.mean()
class_feature_means

Luego, conectamos los vectores mediosy obtenemos la matriz de dispersión intraclase.

In [ ]:
within_class_scatter_matrix = np.zeros((13,13))
for c, rows in df.groupby('class'):
  rows = rows.drop(['class'], axis=1)
    
s = np.zeros((13,13))
for index, row in rows.iterrows():
  x, mc = row.values.reshape(13,1), class_feature_means[c].values.reshape(13,1)
  s += (x - mc).dot((x - mc).T)
  within_class_scatter_matrix += s

A continuación, calculamos la matriz de dispersión entre clases : 

In [ ]:
feature_means = df.mean()
between_class_scatter_matrix = np.zeros((13,13))
for c in class_feature_means:    
    n = len(df.loc[df['class'] == c].index)
    mc, m = class_feature_means[c].values.reshape(13,1), feature_means.values.reshape(13,1)
    between_class_scatter_matrix += n * (mc - m).dot((mc - m).T)

Luego, resolvemos el problema de valores propios generalizados para obtener los discriminantes lineales.

In [ ]:
eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(within_class_scatter_matrix).dot(between_class_scatter_matrix))

Los vectores propios con los valores propios más altos contienen la mayor cantidad de información sobre la distribución de los datos. Por lo tanto, ordenamos los autovalores de mayor a menor y seleccionamos los primeros k autovectores. Para asegurarnos de que el valor propio se corresponda con el mismo vector propio después de la clasificación, los colocamos en una matriz temporal.

In [ ]:
pairs = [(np.abs(eigen_values[i]), eigen_vectors[:,i]) for i in range(len(eigen_values))]
pairs = sorted(pairs, key=lambda x: x[0], reverse=True)
for pair in pairs:
    print(pair[0])

Con solo mirar los valores, es difícil determinar qué parte de la varianza se explica por cada componente. Por tanto, lo expresamos como porcentaje.

In [ ]:
eigen_value_sums = sum(eigen_values)
print('Explained Variance')
for i, pair in enumerate(pairs):
    print('Eigenvector {}: {}'.format(i, (pair[0]/eigen_value_sums).real))

Primero, creamos una matriz W con los dos primeros autovectores.

In [ ]:
w_matrix = np.hstack((pairs[0][1].reshape(13,1), pairs[1][1].reshape(13,1))).real

Luego, guardamos el producto escalar de X y W en una nueva matriz Y : Y = XW

donde X es una matriz n × d con n muestras y d dimensiones, e Y es una matriz n × k con n muestras y k (k < n) dimensiones. En otras palabras, Y se compone de los componentes LDA, o dicho de otra forma, el nuevo espacio de características.

In [ ]:
X_lda = np.array(X.dot(w_matrix))

matplotlib no puede manejar variables categóricas directamente. Por lo tanto, codificamos cada clase como un número para poder incorporar las etiquetas de clase en nuestro gráfico.

In [ ]:
le = LabelEncoder()
y = le.fit_transform(df['class'])

Luego, graficamos los datos en función de los dos componentes LDA y usamos un color diferente para cada clase.

In [ ]:
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.scatter(
    X_lda[:,0],
    X_lda[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)

En lugar de implementar el algoritmo de Análisis Discriminante Lineal desde cero cada vez, podemos usar la clase LinearDiscriminantAnalysis predefinida que pone a nuestra disposición la biblioteca scikit-learn.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_lda = lda.fit_transform(X, y)

Podemos acceder a la siguiente propiedad para obtener la varianza explicada por cada componente.

In [ ]:
lda.explained_variance_ratio_


Al igual que antes, graficamos los dos componentes LDA.

In [ ]:
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.scatter(
    X_lda[:,0],
    X_lda[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)

---
A continuación, echemos un vistazo a cómo se compara LDA con el análisis de componentes principales o PCA. Comenzamos creando y ajustando una instancia de la clase PCA.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X, y)

Podemos acceder a la propiedad explained_variance_ratio_  para ver el porcentaje de la varianza explicada por cada componente.

In [ ]:
pca.explained_variance_ratio_


Como podemos ver, PCA seleccionó los componentes que resultarían en la mayor dispersión (retener la mayor cantidad de información) y no necesariamente los que maximizan la separación entre clases.

In [ ]:
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.scatter(
    X_pca[:,0],
    X_pca[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)

A continuación, veamos si podemos crear un modelo para clasificar el uso de los componentes LDA como características. Primero, dividimos los datos en conjuntos de entrenamiento y prueba.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_lda, y, random_state=1)

Luego, construimos y entrenamos un árbol de decisiones. Después de predecir la categoría de cada muestra en el conjunto de prueba, creamos una matriz de confusión para evaluar el desempeño del modelo.

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
confusion_matrix(y_test, y_pred)

Como podemos ver, el clasificador Decision Tree clasificó correctamente todo en el conjunto de prueba.